In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import warnings
from dev import *
from dev.aggregate import clear_aggregated_cache
from concurrent.futures import ProcessPoolExecutor, as_completed

warnings.filterwarnings("ignore")
clear_aggregated_cache()


# Worker function
def worker(eval_setup, dataset_name, source_name, attack_name, attack_strength):
    performance = get_performance(
        dataset_name,
        source_name,
        attack_name,
        attack_strength,
        mode=eval_setup,
    )
    quality = get_quality(
        dataset_name,
        source_name,
        attack_name,
        attack_strength,
        mode=eval_setup,
    )
    record = {
        "eval_setup": eval_setup,
        "dataset_name": dataset_name,
        "source_name": source_name,
        "attack_name": attack_name,
        "attack_strength": attack_strength,
    }
    record.update({k: v for k, v in performance.items()})
    record.update({k: v for k, v in quality.items()})
    return record


# Configuration for ThreadPoolExecutor
max_workers = 16  # Set the maximum number of workers
job_limit = float("inf")  # Set the job limit, can be set to float('inf') for no limit

# Prepare data for job submission
json_dict = get_all_json_paths(
    lambda _dataset_name, _attack_name, _attack_strength, _source_name, _result_type: (
        (_attack_name is not None)
        and (_attack_strength is not None)
        and (not _source_name.startswith("real"))
    )
)
record_keys = sorted(
    list(set([(key[0], key[3], key[1], key[2]) for key in json_dict.keys()])),
    key=lambda x: tuple((item is None, item) for item in x),
)
total = min(len(EVALUATION_SETUPS) * len(record_keys), job_limit)

# Parallel execution
records = []
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    for eval_setup in EVALUATION_SETUPS.keys():
        for dataset_name, source_name, attack_name, attack_strength in record_keys:
            if len(futures) < job_limit:
                future = executor.submit(
                    worker,
                    eval_setup,
                    dataset_name,
                    source_name,
                    attack_name,
                    attack_strength,
                )
                futures.append(future)
            else:
                break

    # Collect results
    for future in tqdm(as_completed(futures), total=total):
        records.append(future.result())

# Convert to DataFrame and save
table = pd.DataFrame(records)
table.to_pickle("all_result_v1.pkl")

  0%|          | 0/3459 [00:00<?, ?it/s]

In [3]:
pd.read_pickle("all_result_v1.pkl")

,eval_setup,dataset_name,source_name,attack_name,attack_strength,acc_1,auc_1,low100_1,low1000_1,legacy_fid,clip_fid,psnr,ssim,nmi,lpips,watson,aesthetics,artifacts,clip_score
0,removal,dalle3,stable_sig,4x_regen_kl_vae,8.0,0.5001,0.360216,0.0016,0.0004,"(46.12348781608932, 0.0)","(8.992265823301551, 0.0)","(17.199796537364293, 2.269189505607515)","(0.39733862463659614, 0.09261186435461086)","(1.0911852889505835, 0.031366562294090135)","(0.4905354205071926, 0.10838788701403224)","(6491883.1755, 13013495.7750652)","(1.281013566672802, 0.8179406189703678)","(-0.4907000326514244, 0.40590312896105485)","(-0.02172722030710429, 0.019664222858970473)"
1,removal,dalle3,stable_sig,adv_emb_clip_untg_alphaRatio_0.05_step_200,2.0,0.9984,0.999695,0.9980,0.9972,"(1.5416575532917705, 0.0)","(25.287483213360108, 3.552713678800501e-15)","(43.71427362464086, 0.3420215417855504)","(0.9867237320919474, 0.005671492383389793)","(1.6133441606600623, 0.020300613091421078)","(0.008419898152875247, 0.008218121675276056)","(437109.038390625, 3971476.2536129635)","(0.15220061630010606, 0.24954178237096644)","(-0.05379353821277619, 0.12816892088923812)","(-0.0012738871546462177, 0.006560638785684172)"
2,removal,dalle3,stable_sig,2x_regen,20.0,0.5298,0.537489,0.0098,0.0018,"(4.808336591091461, 0.0)","(0.38104822151503015, 0.0)","(23.249902572912536, 3.1788729755903278)","(0.6833927196897406, 0.10811329418038841)","(1.187487968029458, 0.059042823517310404)","(0.12361973528191447, 0.04310015426425451)","(3685886.7109375, 35441197.346991636)","(0.0821058354973793, 0.3396569703025583)","(0.015196512311697006, 0.1788275712807598)","(-0.0010460978876799345, 0.008120075928693423)"
3,removal,dalle3,stable_sig,adv_cls_real_wm_0.01_50_warm,8.0,0.9980,0.999471,0.9980,0.9964,"(0.7113836169586989, 0.0)","(0.22029766436683929, 0.0)","(42.31186376428473, 1.8866201639732538)","(0.983585539693678, 0.00516051468205318)","(1.558867287817048, 0.043258092467638884)","(0.010971170436474494, 0.008905676611333534)","(538782.2018359374, 7158327.205991846)","(0.031158906877040864, 0.16183392432548427)","(0.0009671718478202819, 0.09072956191395361)","(-0.00048506836034357547, 0.004469317320971539)"
4,removal,dalle3,stable_sig,adv_cls_real_wm_0.01_50_warm,6.0,0.9981,0.999578,0.9980,0.9966,"(0.47755575898213465, 0.0)","(0.14674711078012592, 0.0)","(43.936693335794004, 1.3308611051112806)","(0.9882749988797969, 0.0037089530882295657)","(1.6020914021720603, 0.030189437035218994)","(0.007419525931501994, 0.0066372411923751595)","(362634.86104296875, 3523992.446032149)","(0.02350645011663437, 0.1415184981726149)","(0.0008609980940818787, 0.08250103754400105)","(-0.00015730367321521043, 0.003923573073218316)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3454,combined,mscoco,tree_ring,regen_diffusion_prompt,160.0,0.9619,0.993320,0.9006,0.7336,"(8.586375371709012, 1.4043333874306805e-15)","(1.9254719088750103, 3.510833468576701e-16)","(21.71826298577528, 2.872949031655825)","(0.6279947179276391, 0.12845373357061987)","(1.1643855177796882, 0.05076783329707679)","(0.17630047589819878, 0.04575402362679158)","(8850697.016785156, 329854881.56977266)","(0.48193047654628757, 0.5125005902952873)","(-0.284070571795106, 0.2799417479842635)","(-0.0009336986921261996, 0.010856786538976055)"
3455,combined,mscoco,tree_ring,regen_vae,5.0,0.9984,0.999987,0.9996,0.9974,"(6.674961939260414, 1.2560739669470201e-15)","(2.0396622313477053, 3.1401849173675503e-16)","(33.22537572641761, 2.76863357625072)","(0.9219745583493202, 0.033548934866993715)","(1.3692031110431717, 0.06330696905678053)","(0.09298591066687367, 0.0522560009264153)","(549703.8531171875, 392775.0001276065)","(0.24785685396194457, 0.3224470661981921)","(-0.19860438038408756, 0.20142869003351288)","(-0.00596388701943215, 0.008134300972922613)"
3456,combined,mscoco,tree_ring,regen_diffusion_prompt,80.0,0.9829,0.998044,0.9742,0.9206,"(6.597553532255489, 6.280369834735101e-16)","(1.3471922281754232, 1.570092458683775